# Aufgabe

In dem Aufgaben finden Sie 2 Dateien. Die Dateien enthalten englische Reviews von Filmen. Die eine Datei enthält negative die andere positive Reviews. 

Wir wollen einen Klassifikator bauen, der vorhersagt, ob ein Review positiv oder negativ ist. Machen Sie dazu folgendes:

1. Teilen Sie die Daten in Test und Trainingsdaten ein.
2. Extrahieren Sie alle Adjektive, Verben und Adverbien aus den Reviews
3. Nutzen Sie alle Adjektive, Verben und Adverbien, die mindesten 5 Mal vorkommen als Merkmale und Trainieren Sie einen Klassifikator, der logistische Regression nutzt.
5. Evaluieren Sie den Klassifikator

# Lösung

In [2]:
import codecs 

def readtext(dateiname):
    text = ''
    d = codecs.open(dateiname,'r','utf8')
    for zeile in d:
        text += str(zeile)
    d.close()

    return text

In [3]:
import glob

def read_data(directories):
    docs = []
    for directory in directories:
        for file in glob.glob(directory+"/*.txt"):
            text = readtext(file)
            docs.append((text,directory))
    return docs

data = read_data(['neg','pos'])

In [4]:
import random

random.shuffle(data)
train_data_raw = data[80:]
test_data_raw = data[:80]

In [5]:
import nltk
import re
from collections import Counter

stopwords = nltk.corpus.stopwords.words('english')

def features_from_text(text, POSlist):
    wordcounts = Counter()
    tlen = 0
    
    sentlist = nltk.sent_tokenize(text,language='english')
    for sent in sentlist:
        tokens = nltk.word_tokenize(sent,language='english')
        tokens = [lemma for (lemma,pos) in nltk.pos_tag(tokens) if pos in POSlist]
        wordcounts.update(tokens)
        tlen +=len(tokens)
    return {w:wordcounts[w]/tlen for w in wordcounts}

In [6]:
# poslist = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
# poslist = ['RB', 'RBR', 'RBS']
poslist = ['JJ', 'JJR', 'JJS']


test_data = [(features_from_text(text,poslist),cl) for text,cl in test_data_raw]
train_data = [(features_from_text(text,poslist),cl) for text,cl in train_data_raw]

In [7]:
train_data[13]

({'alex': 0.023529411764705882,
  'tony': 0.011764705882352941,
  'next': 0.011764705882352941,
  'last': 0.011764705882352941,
  'hard': 0.03529411764705882,
  'clenched': 0.011764705882352941,
  'strychnine-laced': 0.011764705882352941,
  'other': 0.011764705882352941,
  'psychedelic': 0.011764705882352941,
  'leisurely': 0.011764705882352941,
  'ferris': 0.011764705882352941,
  'steady': 0.011764705882352941,
  'reassuring': 0.011764705882352941,
  'short': 0.011764705882352941,
  'smooth': 0.011764705882352941,
  'refined': 0.011764705882352941,
  'acid': 0.023529411764705882,
  'full': 0.011764705882352941,
  'aggressive': 0.011764705882352941,
  'grotesque': 0.011764705882352941,
  'comic': 0.023529411764705882,
  'twisted': 0.011764705882352941,
  'pharmacological': 0.011764705882352941,
  'well-made': 0.011764705882352941,
  'mixed': 0.011764705882352941,
  'true': 0.011764705882352941,
  'self-professed': 0.011764705882352941,
  'gonzo': 0.011764705882352941,
  'johnny': 0.035

In [8]:
docfreq = Counter()
for (wfreq,c) in train_data:
    docfreq.update(wfreq.keys())

In [9]:
docfreq.most_common(10)

[('good', 1101),
 ('other', 1067),
 ('more', 976),
 ('i', 873),
 ('first', 867),
 ('little', 813),
 ('much', 796),
 ('many', 755),
 ('best', 735),
 ('bad', 728)]

In [10]:
from sklearn import linear_model, datasets

allfeatures = [w for w in docfreq if docfreq[w] > 4]

def make_feat_vec(featmap,featlist):
    vec = []
    for f in featlist:
        vec.append(featmap.get(f,0.0))
    return vec

train_vec =  [make_feat_vec(feats,allfeatures) for feats,cls in train_data]
train_label = [cls for feats,cls in train_data]

In [11]:
train_vec[86]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.011904761904761904,
 0.0,
 0.0,
 0.011904761904761904,
 0.0,
 0.05952380952380952,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.011904761904761904,
 0.0,
 0.0,
 0.011904761904761904,
 0.0,
 0.0,
 0.0,
 0.011904761904761904,
 0.0,
 0.0,
 0.0,
 0.011904761904761904,
 0.0,
 0.011904761904761904,
 0.011904761904761904,
 0.0,
 0.0,
 0.0,
 0.0,
 0.011904761904761904,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.023809523809523808,
 0.023809523809523808,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.011904761904761904,
 0.0,
 0.0,
 0.0,
 0.0,
 0.011904761904761904,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.023809523809523808,
 0.0,
 0.0,
 0.0,
 0

In [12]:
train_label[86]

'neg'

In [13]:
logreg = linear_model.LogisticRegression(C=1e9,verbose=True)
logreg.fit(train_vec,train_label)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s finished


LogisticRegression(C=1000000000.0, verbose=True)

In [14]:
test_vec = [make_feat_vec(feats,allfeatures) for feats,cls in test_data]
test_label = [cls for feats,cls in test_data]

pred_label = list(logreg.predict(test_vec))

In [15]:
correct = 0
for i in range(len(test_label)):
    if test_label[i] == pred_label[i]:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/len(test_label)))

85.0 Prozent korrekt
